In [10]:
import torch.optim as optim
import torch.nn.functional as F

import bayescache.api as api
from bayescache.models import mtcnn
from bayescache.data.synthetic import Synthetic
from bayescache.api.source import TrainingData
from bayescache.callbacks.time_tracker import TimeTracker

In [2]:
hparams = mtcnn.Hyperparameters()
# Update hyperparameters for the Synthetic data.
hparams.vocab_size = 4014
hparams.max_sent_len = 1500

model = mtcnn.new(hparams)

optimizer = optim.RMSprop(model.parameters(), lr=7.0e-4, eps=1e-3)

In [3]:
learner = api.Learner(device='cpu', model=model)
#learner.summary()

In [4]:
train = Synthetic(root='/Users/yngtodd/data', partition='train', download=True)
test = Synthetic(root='/Users/yngtodd/data', partition='test', download=True)

source = TrainingData(train_source=train, val_source=test, num_workers=2, batch_size=1)

In [5]:
metrics = learner.metrics()
callbacks = [TimeTracker()]

training_info = api.TrainingInfo(
    start_epoch_idx=0,
    run_name='test',
    metrics=metrics,
    callbacks=callbacks
)

In [6]:
training_info.on_train_begin()

In [ ]:
for global_epoch_idx in range(training_info.start_epoch_idx + 1, 1 + 1):
    epoch_info = api.EpochInfo(
        training_info=training_info,
        global_epoch_idx=global_epoch_idx,
        batches_per_epoch=source.train_iterations_per_epoch(),
        optimizer=optimizer
    )

    # Execute learning
    learner.run_epoch(epoch_info, source)

### Diagnosing Loss

In [28]:
import torch
from torch.utils.data import DataLoader

In [8]:
trainloader = DataLoader(train)

In [118]:
for idx, (data, label) in enumerate(trainloader):
    logits = model(data)
    #print(len(logits))
    #print(logits)
    print(len(label))
    print(label.shape)
    #F.cross_entropy(logits[0], label[0])
    if idx == 0:
        break 

1
torch.Size([1, 4])


In [119]:
logits

[tensor([[ 1.6869, -1.9427,  1.3464, -1.7673,  1.4861,  1.2414]],
        grad_fn=<AddmmBackward>),
 tensor([[-0.3516,  1.7404]], grad_fn=<AddmmBackward>),
 tensor([[-1.3831, -0.1916]], grad_fn=<AddmmBackward>),
 tensor([[-1.2198,  0.4501,  1.7425]], grad_fn=<AddmmBackward>)]

In [120]:
label

tensor([[14,  1,  2,  2]])

In [132]:
F.cross_entropy(F.softmax(logits[0]), torch.tensor((5,)))

/Users/yngtodd/src/checkout/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor(1.7667, grad_fn=<NllLossBackward>)

In [117]:
logits

[tensor([[ 1.6869, -1.9427,  1.3464, -1.7673,  1.4861,  1.2414]],
        grad_fn=<AddmmBackward>),
 tensor([[-0.3516,  1.7404]], grad_fn=<AddmmBackward>),
 tensor([[-1.3831, -0.1916]], grad_fn=<AddmmBackward>),
 tensor([[-1.2198,  0.4501,  1.7425]], grad_fn=<AddmmBackward>)]

In [115]:
logits[0].shape
x = label[0][0].item()
target = torch.tensor((x,))
target

tensor([14])

In [106]:
input = torch.randn(1, 6, requires_grad=True)
target = torch.randint(6, (1,), dtype=torch.int64)

In [107]:
input.shape
target

tensor([3])

### Shape of Things to Come:

In [121]:
data, labels = train.load_data()

In [122]:
len(labels)

8000

In [123]:
len(labels[:,0])

8000

In [129]:
import numpy as np
unq, counts = np.unique(labels[:,0], return_counts=True)

In [131]:
unq

array([ 1,  3,  5, 10, 13, 14])

In [126]:
counts

array([1, 2, 1])

In [ ]:
data[0].shape

In [ ]:
sum(counts)

In [88]:
data.shape

torch.Size([1, 1500])

In [96]:
labels

array([[14,  1,  2,  2],
       [13,  1,  2,  1],
       [14,  1,  1,  0],
       ...,
       [14,  1,  2,  1],
       [14,  2,  2,  1],
       [14,  1,  2,  1]])

In [95]:
data

array([[  35,  197,  232, ...,    0,    0,    0],
       [ 631, 1870,  427, ...,    0,    0,    0],
       [  79,  291,   96, ...,    0,    0,    0],
       ...,
       [3182,  196,  719, ...,    0,    0,    0],
       [  93,  119,  109, ...,    0,    0,    0],
       [  90,  414,   41, ...,    0,    0,    0]])